## Import Libraries

In [1]:
'''basics'''
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import vectorize_embed as em
import pandas as pd
#import pickle5 as pickle
import pickle
import numpy as np


'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB


'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, multi_labeled
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings('ignore')

from tabulate import tabulate  

import joblib

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


# Forest

## Import data and holdout data for prediction

In [2]:
df = pd.read_excel('../../data/processed/encoded_labels/Forest.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'forest', 'tropical_forests', 'temperate_forests', 'dryland_forests',
       'montane_forests', 'intact_forests', 'boreal_forests_taiga_forests'],
      dtype='object')

##  Features engineering

In [3]:
categories = ['forest', 'tropical_forests', 'temperate_forests', 'dryland_forests',
       'montane_forests', 'intact_forests', 'boreal_forests_taiga_forests']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    

In [4]:
header = lis[0].keys()
rows =  [x.values() for x in lis]
print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                    </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>forest                      </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>tropical_forests            </td><td>Roberta    
 Bert    
 Distilbert    
 Glove              </td><td>0.6666666666666666    
 0.6388888888888888    
 0.6388888888888888    
 0.3611111111111111                 </td><td>0.6563467492260062   
 0.6388888888888888   
 0.5833333333333334   
 0.53125                  </td><td>0.6836363636363636    
 0.6636363636363636    
 0.5872727272727273    
 0.5145454545454545               </td><td>0.6493506493506493    
 0.624699

# conserved_areas

In [5]:
df = pd.read_excel('../../data/processed/encoded_labels/Conserved_Areas.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'conserved_areas', 'marine_and_coastal_protected_areas',
       'terrestrial_protected_areas',
       'indigenous_and_communities_conserved_areas_icca',
       'transboundary_conservation_areas', 'productive_landscapes_seascapes',
       'key_biodiversity_areas_kba',
       '_important_bird_and_biodiversity_areas_ibas',
       'specially_protected_areas_spas', 'protected_areas_network',
       'oecm_other_effective_area_based_conservation_measures',
       'locally_managed_marine_areas'],
      dtype='object')

In [6]:
categories = ['conserved_areas', 'marine_and_coastal_protected_areas',
       'terrestrial_protected_areas',
       'indigenous_and_communities_conserved_areas_icca',
       'transboundary_conservation_areas', 'productive_landscapes_seascapes',
       'key_biodiversity_areas_kba',
       '_important_bird_and_biodiversity_areas_ibas',
       'specially_protected_areas_spas', 'protected_areas_network',
       'oecm_other_effective_area_based_conservation_measures',
       'locally_managed_marine_areas']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    



In [7]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                                             </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>conserved_areas                                      </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>marine_and_coastal_protected_areas                   </td><td>Distilbert    
 Bert    
 Roberta    
 Glove              </td><td>0.8913043478260869    
 0.8478260869565217    
 0.8695652173913043    
 0.8260869565217391                 </td><td>0.8063063063063063   
 0.7441558441558441   
 0.7833333333333333   
 0.7205882352941176                  </td><td>0.8355263157894737    
 0.8092105263157895    
 0.7236842105

# Freshwater

In [8]:
df = pd.read_excel('../../data/processed/encoded_labels/Freshwater.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'freshwater', 'rivers_and_river_basins', 'lakes', 'aquifers',
       'estuaries'],
      dtype='object')

In [9]:
categories = ['freshwater', 'rivers_and_river_basins', 'lakes', 'aquifers',
       'estuaries']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)


In [10]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category               </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>freshwater             </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>rivers_and_river_basins</td><td>Roberta    
 Glove    
 Bert    
 Distilbert              </td><td>0.8888888888888888    
 0.7222222222222222    
 0.6666666666666666    
 0.4444444444444444                 </td><td>0.8888888888888888   
 0.8214285714285714   
 0.7076923076923077   
 0.44155844155844154                  </td><td>0.8888888888888888    
 0.7222222222222222    
 0.6666666666666666    
 0.4444444444444444               </td><td>0.8888888888888888    
 0.698996655

# Grassland

In [11]:
df = pd.read_excel('../../data/processed/encoded_labels/Grassland.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'grassland', 'grazing_land', 'tropical_grassland',
       'temperate_grassland', 'savanna', 'steppes', 'drylands'],
      dtype='object')

In [12]:


categories = ['grassland', 'grazing_land', 'tropical_grassland',
       'temperate_grassland', 'savanna', 'steppes', 'drylands']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    


In [13]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category           </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>grassland          </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>grazing_land       </td><td>Distilbert    
 Roberta    
 Bert    
 Glove              </td><td>0.6666666666666666    
 0.6    
 0.5333333333333333    
 0.4666666666666667                 </td><td>0.6666666666666666   
 0.65   
 0.5555555555555556   
 0.4732142857142857                  </td><td>0.611111111111111    
 0.6388888888888888    
 0.5555555555555556    
 0.4722222222222222               </td><td>0.6031746031746033    
 0.5982142857142857    
 0.5333333333333333    
 0.4642

# Desert

In [14]:
df = pd.read_excel('../../data/processed/encoded_labels/Desert.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'desert', 'sub_tropical_hot_and_dry_desert', 'rift_valley',
       'semi_arid_cold_winter_desert', 'coast_desert'],
      dtype='object')

In [15]:

categories = ['desert', 'sub_tropical_hot_and_dry_desert', 'rift_valley',
       'semi_arid_cold_winter_desert', 'coast_desert']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    


In [16]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                       </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>desert                         </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>sub_tropical_hot_and_dry_desert</td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>0.8    
 0.8    
 0.8    
 0.8                 </td><td>0.4   
 0.4   
 0.4   
 0.4                  </td><td>0.5    
 0.5    
 0.5    
 0.5               </td><td>0.4444444444444445    
 0.4444444444444445    
 0.4444444444444445    
 0.4444444444444445           </td></tr>
<tr><td>rift_valley                    </td><td>Glove    
 Distilbert    
 Rober

# Marine

In [17]:
df = pd.read_excel('../../data/processed/encoded_labels/Marine.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'marine', 'seas', 'coasts', 'coral_reefs', 'seagrasses',
       'large_marine_ecosystem', 'exclusive_economic_zone',
       'areas_beyond_national_jurisdiction'],
      dtype='object')

In [18]:

categories = ['marine', 'seas', 'coasts', 'coral_reefs', 'seagrasses',
       'large_marine_ecosystem', 'exclusive_economic_zone',
       'areas_beyond_national_jurisdiction']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    

In [19]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category                          </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>marine                            </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>seas                              </td><td>Bert    
 Roberta    
 Glove    
 Distilbert              </td><td>0.8461538461538461    
 0.8076923076923077    
 0.8846153846153846    
 0.7307692307692307                 </td><td>0.6231884057971014   
 0.5795454545454546   
 0.4423076923076923   
 0.4318181818181818                  </td><td>0.6231884057971014    
 0.6014492753623188    
 0.5    
 0.41304347826086957               </td><td>0.623188405797101

# Wetlands

In [20]:
df = pd.read_excel('../../data/processed/encoded_labels/Wetlands.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'wetlands', 'mangroves', 'marshes', 'swamps', 'peatlands'],
      dtype='object')

In [21]:

categories = ['wetlands', 'mangroves', 'marshes', 'swamps', 'peatlands']
#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    

In [22]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category  </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>wetlands  </td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>mangroves </td><td>Bert    
 Glove    
 Roberta    
 Distilbert              </td><td>0.8333333333333334    
 0.6666666666666666    
 0.5    
 0.5833333333333334                 </td><td>0.8888888888888888   
 0.6666666666666666   
 0.4857142857142857   
 0.2916666666666667                  </td><td>0.8    
 0.6285714285714286    
 0.4857142857142857    
 0.5               </td><td>0.8125    
 0.625    
 0.48571428571428577    
 0.3684210526315789           </td></tr>
<tr><td>marshes   </td><td>Glove

# Human_altered_areas

In [23]:
df = pd.read_excel('../../data/processed/encoded_labels/Human_Altered_Areas.xlsx')
df.columns

Index(['Unnamed: 0', 'PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'human_altered_areas', 'urban_area', 'rural_area', 'mining_site',
       'industrial_site', 'heritage_site', 'entry_exit_ports',
       'contaminated_site'],
      dtype='object')

In [24]:

categories = ['human_altered_areas', 'urban_area', 'rural_area', 'mining_site',
       'industrial_site', 'heritage_site', 'entry_exit_ports',
       'contaminated_site']

#Turning the labels into numbers
y = pd.DataFrame(df, columns = categories)

X = df['all_text_clean'].astype('str').tolist()
#Creating a dict of the embeddings
embedding_dict = {'Glove' : 'average_word_embeddings_glove.6B.300d', 
                  'Distilbert':'distilbert-base-nli-mean-tokens', 
                  'Roberta' : 'roberta-base-nli-stsb-mean-tokens', 
                  'Bert' : 'bert-base-nli-stsb-mean-tokens'}
              
sgd_classifier = SGDClassifier(alpha=1e-06,
                               loss='log',
                               max_iter=1000,
                               penalty='l1',
                               random_state = 3,
                               tol=0.001)
model = OneVsRestClassifier(sgd_classifier)

#Train test split with stratified sampling for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .3, 
                                                    shuffle = True,  
                                                    random_state = 3)
y_train = y_train.dropna(axis=1)

#Function to get the scores for each model in a df
def model_score_df(embedding_dict, X_train, X_test, y_train, y_test, category):   
    embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in embedding_dict.items():   
        embedding_name.append(k)   
        model.fit(em.get_embeddings(v, X_train), y_train)
        
        # save the model to disk
        filename = '../'+category+'_'+k+'model.sav'
        joblib.dump(model, filename)
        
        y_pred = model.predict(em.get_embeddings(v, X_test))
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([embedding_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['embedding_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

      
lis = []
for category in categories:
    dic = {}
    dff = model_score_df(embedding_dict, X_train, X_test, y_train[category], y_test[category], category)
    # Using DataFrame.insert() to add a column
    dic['Category'] = category
    dic['Classifiers'] = '    \n '.join(dff.embedding_name.apply(str).tolist())
    dic['accuracy_score'] = '    \n '.join(dff.accuracy_score.apply(str).tolist()) 
    dic['precision_score'] = '   \n '.join(dff.precision_score.apply(str).tolist())
    dic['recall_score'] = '    \n '.join(dff.recall_score.apply(str).tolist())
    dic['f1_score'] = '    \n '.join(dff.f1_score.apply(str).tolist())
    lis.append(dic)
    

In [25]:
header = lis[0].keys()
rows =  [x.values() for x in lis]

print(tabulate(rows, header, tablefmt='html'))

<table>
<thead>
<tr><th>Category           </th><th>Classifiers  </th><th>accuracy_score  </th><th>precision_score  </th><th>recall_score  </th><th>f1_score  </th></tr>
</thead>
<tbody>
<tr><td>human_altered_areas</td><td>Glove    
 Distilbert    
 Roberta    
 Bert              </td><td>1.0    
 1.0    
 1.0    
 1.0                 </td><td>1.0   
 1.0   
 1.0   
 1.0                  </td><td>1.0    
 1.0    
 1.0    
 1.0               </td><td>1.0    
 1.0    
 1.0    
 1.0           </td></tr>
<tr><td>urban_area         </td><td>Bert    
 Roberta    
 Distilbert    
 Glove              </td><td>0.7910447761194029    
 0.746268656716418    
 0.7611940298507462    
 0.7910447761194029                 </td><td>0.6915708812260537   
 0.6293800539083558   
 0.637987012987013   
 0.7032258064516128                  </td><td>0.6282051282051282    
 0.6230769230769231    
 0.6089743589743589    
 0.5807692307692308               </td><td>0.6446969696969697    
 0.625944170771757    
 0.6